### Bibliotecas

In [185]:
import pandas as pd
import numpy as np

### Dataset

In [186]:
#Carregar o dataset Titanic
df_train = pd.read_csv(r'C:\Python\Titanic\train.csv')

In [187]:
# Exibir as 5 primeiras linhas do dataset
print(df_train.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [188]:
# Exibir informações gerais sobre o DataFrame
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [189]:
# Exibir estatísticas descritivas para as colunas numéricas
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [190]:
# Contar valores ausentes por coluna
print(df_train.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [191]:
# Quantidade de sobreviventes
print("Quantidade de sobreviventes:", df_train['Survived'].value_counts(normalize=True))


Quantidade de sobreviventes: Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64


In [192]:
# Exibir a quantidade de valores ausentes por coluna
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### Tratamento dos Dados

In [193]:
# Excluir a coluna 'Cabin'
df_train.drop('Cabin', axis=1, inplace=True)

# Exibir a novo dataset
print(df_train.head())
print(df_train.info())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Embarked  
0      0         A/5 21171   7.2500        S  
1      0          PC 17599  71.2833        C  
2      0  STON/O2. 3101282   7.9250        S  
3      0            113803  53.1000        S  
4      0            373450   8.0500        S  
<class 'pandas.core.frame.DataFrame'>


In [194]:
# Resolvendo o problema de valores ausentes na coluna 'Embarked'
# 1. Calcular a moda da coluna 'Embarked'
moda_embarked = df_train['Embarked'].mode()[0] # [0] para pegar o primeiro valor se houver múltiplas modas
print(f"A moda da coluna 'Embarked' é: {moda_embarked}")

# 2. Preencher os valores ausentes (NaNs) na coluna 'Embarked' com a moda calculada
df_train['Embarked'].fillna(moda_embarked, inplace=True)

# 3. Verificar se os valores ausentes foram preenchidos (boa prática)
print("\nContagem de valores ausentes após preencher 'Embarked':")
print(df_train.isnull().sum())

A moda da coluna 'Embarked' é: S

Contagem de valores ausentes após preencher 'Embarked':
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         0
dtype: int64


C:\Users\NVME\AppData\Local\Temp\ipykernel_11448\2126310383.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['Embarked'].fillna(moda_embarked, inplace=True)


In [195]:
# 4. Agora sim, calcular a média da coluna 'Age'
media_idade = df_train['Age'].mean()
print(f"A média da coluna 'Age' é: {media_idade:.2f}")

# 5. Calcular a mediana da coluna 'Age'
mediana_idade = df_train['Age'].median()
print(f"A mediana da coluna 'Age' é: {mediana_idade:.2f}")


A média da coluna 'Age' é: 29.70
A mediana da coluna 'Age' é: 28.00


In [196]:
# Resolvendo o problema de valores ausentes na coluna 'Age'
# 1. Preencher os valores ausentes (NaNs) na coluna 'Age' com a moda calculada
df_train['Age'].fillna(mediana_idade, inplace=True)

# 2. Verificar se os valores ausentes foram preenchidos (boa prática)
print("\nContagem de valores ausentes após preencher 'Age':")
print(df_train.isnull().sum())


Contagem de valores ausentes após preencher 'Age':
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


C:\Users\NVME\AppData\Local\Temp\ipykernel_11448\3576810113.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['Age'].fillna(mediana_idade, inplace=True)


### Insight Sobre a Relação entre Sexo e Sobreviventes

In [197]:
# Calcular a contagem de sobreviventes e não-sobreviventes para cada sexo
print("Contagem de sobreviventes (1) e não-sobreviventes (0) por Sexo:")
print(df_train.groupby('Sex')['Survived'].value_counts())

Contagem de sobreviventes (1) e não-sobreviventes (0) por Sexo:
Sex     Survived
female  1           233
        0            81
male    0           468
        1           109
Name: count, dtype: int64


In [198]:
# Para ter uma visão mais completa, vamos também calcular a proporção
print("\nProporção de sobreviventes (1) e não-sobreviventes (0) por Sexo:")
print(df_train.groupby('Sex')['Survived'].value_counts(normalize=True))


Proporção de sobreviventes (1) e não-sobreviventes (0) por Sexo:
Sex     Survived
female  1           0.742038
        0           0.257962
male    0           0.811092
        1           0.188908
Name: proportion, dtype: float64


In [199]:
# Vamos calcular a proporção de mulheres e homens DENTRO do grupo de sobreviventes

# Filtrar o DataFrame para incluir apenas os sobreviventes
survivors_df = df_train[df_train['Survived'] == 1]

# Calcular a proporção de sexo dentro do grupo de sobreviventes
print("Proporção de Mulheres e Homens entre os Sobreviventes:")
print(survivors_df['Sex'].value_counts(normalize=True))

Proporção de Mulheres e Homens entre os Sobreviventes:
Sex
female    0.681287
male      0.318713
Name: proportion, dtype: float64


In [200]:
# Vamos calcular a proporção de mulheres e homens DENTRO do grupo de não-sobreviventes

# Filtrar o DataFrame para incluir apenas os não-sobreviventes
survivors_df = df_train[df_train['Survived'] == 0]

# Calcular a proporção de sexo dentro do grupo de não-sobreviventes
print("Proporção de Mulheres e Homens entre os não-Sobreviventes:")
print(survivors_df['Sex'].value_counts(normalize=True))

Proporção de Mulheres e Homens entre os não-Sobreviventes:
Sex
male      0.852459
female    0.147541
Name: proportion, dtype: float64


### Insight Sobre a Relação entre Classe e Sobreviventes

In [201]:
# Calcular a contagem de sobreviventes e não-sobreviventes para cada classe
print("Contagem de sobreviventes (1) e não-sobreviventes (0) por Classe:")
print(df_train.groupby('Pclass')['Survived'].value_counts())

Contagem de sobreviventes (1) e não-sobreviventes (0) por Classe:
Pclass  Survived
1       1           136
        0            80
2       0            97
        1            87
3       0           372
        1           119
Name: count, dtype: int64


In [202]:
# Para ter uma visão mais completa, vamos também calcular a proporção
print("\nProporção de sobreviventes (1) e não-sobreviventes (0) por Classe:")
print(df_train.groupby('Pclass')['Survived'].value_counts(normalize=True))


Proporção de sobreviventes (1) e não-sobreviventes (0) por Classe:
Pclass  Survived
1       1           0.629630
        0           0.370370
2       0           0.527174
        1           0.472826
3       0           0.757637
        1           0.242363
Name: proportion, dtype: float64


In [203]:
# Vamos calcular a proporção das 3 classes DENTRO do grupo de sobreviventes

# Filtrar o DataFrame para incluir apenas os sobreviventes
survivors_df = df_train[df_train['Survived'] == 1]

# Calcular a proporção de classe dentro do grupo de sobreviventes
print("Proporção das 3 classes entre os Sobreviventes:")
print(survivors_df['Pclass'].value_counts(normalize=True))

Proporção das 3 classes entre os Sobreviventes:
Pclass
1    0.397661
3    0.347953
2    0.254386
Name: proportion, dtype: float64


In [204]:
# Vamos calcular a proporção das 3 classes DENTRO do grupo de não-sobreviventes

# Filtrar o DataFrame para incluir apenas os não-sobreviventes
survivors_df = df_train[df_train['Survived'] == 0]

# Calcular a proporção de classe dentro do grupo de não-sobreviventes
print("Proporção das 3 classes entre os não-Sobreviventes:")
print(survivors_df['Pclass'].value_counts(normalize=True))

Proporção das 3 classes entre os não-Sobreviventes:
Pclass
3    0.677596
2    0.176685
1    0.145719
Name: proportion, dtype: float64


### Insight Sobre a Relação entre Idade e Sobreviventes

In [ ]:
# Calcular a contagem de sobreviventes e não-sobreviventes para cada classe
print("Contagem de sobreviventes (1) e não-sobreviventes (0) por Classe:")
print(df_train.groupby('Pclass')['Survived'].value_counts())

In [ ]:
# Para ter uma visão mais completa, vamos também calcular a proporção
print("\nProporção de sobreviventes (1) e não-sobreviventes (0) por Classe:")
print(df_train.groupby('Pclass')['Survived'].value_counts(normalize=True))